In [1]:
# BASIC_CBC.IPYNB - A simple case of resource compeition CBC applied to a constritutive gene
# By Kirill Sechkar

# PACKAGE IMPORTS
import numpy as np
import jax
import jax.numpy as jnp
import functools
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio

import time

# CIRCUIT AND EXTERNAL INPUT IMPORTS
from sim_tools.cell_model import *
import sim_tools.genetic_modules as gms
import sim_tools.controllers as ctrls
import sim_tools.reference_switchers as refsws
import sim_tools.ode_solvers as odesols

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

Loading BokehJS ...

In [2]:
# DEFINE CIRCUIT PARAMETERS TO BE CONSIDERED
circ_par={}

# set the parameters for the synthetic genes
circ_par['c_ofp']=100
circ_par['a_ofp']=1000
circ_par['c_ta']=100
circ_par['a_ta']=10
circ_par['c_b']=100
circ_par['a_b']=2000

In [3]:
# SIMULATE THE SETUP WITH CONSTANT INDUCER CONCENTRATIONS

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
cellmodel_par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(cellmodel_par)  # get default initial conditions

# add reference tracker switcher
cellmodel_par_with_refswitch, ref_switcher = cellmodel_auxil.add_reference_switcher(cellmodel_par,   # cell model parameters
                                                                               refsws.no_switching_initialise,    # function initialising the reference switcher
                                                                               refsws.no_switching_switch   # function switching the references to be tracked
                                                                               )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_eff_mRNA, module2_eff_mRNA, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_and_each, synth_miscs_total_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar,\
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = cellmodel_auxil.add_modules_and_controller(
        # module 1
        gms.constfp_initialise,  # function initialising the circuit
        gms.constfp_ode,  # function defining the circuit ODEs
        gms.constfp_F_calc,
        gms.constfp_eff_mRNA,
        # function calculating the circuit genes' transcription regulation functions
        # module 2
        gms.cicc_initialise,  # function initialising the circuit
        gms.cicc_ode,  # function defining the circuit ODEs
        gms.cicc_F_calc,
        gms.cicc_eff_mRNA,
        # function calculating the circuit genes' transcription regulation functions
        # controller
        ctrls.cci_initialise,  # function initialising the controller
        ctrls.cci_action,  # function calculating the controller action
        ctrls.cci_ode,  # function defining the controller ODEs
        ctrls.cci_update,  # function updating the controller based on measurements
        # cell model parameters and initial conditions
        cellmodel_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes= synth_genes_total_and_each[0]
module1_genes = synth_genes_total_and_each[1]
module2_genes = synth_genes_total_and_each[2]
synth_miscs= synth_miscs_total_and_each[0]
module1_miscs = synth_miscs_total_and_each[1]
module2_miscs = synth_miscs_total_and_each[2]

# set the circuit parameters common for all cases
par.update(circ_par)

# set simulation parameters
tf = (0.0, 20.0)  # simulation time frame

# measurement time step
meastimestep = 0.1  # hours
control_delay=0.0
u0=0.0
ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                  control_delay=control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=1e-5)

# inducer levels to be considered
u_range = np.logspace(-1, 2, 10)
ofp_levels_for_u_range = np.zeros(len(u_range))

for i in range(0,len(u_range)):
    # set the inducer concentration
    init_conds['inducer_level'] = u_range[i]
    
    ts_jnp, xs_jnp,\
        ctrl_memorecord_jnp, uexprecord_jnp, \
        refrecord_jnp  = ode_sim(par,   # model parameters
                                 ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                                 odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                                 controller_ctrledvar,    # name of the variable read and steered by the controller
                                 controller_update, controller_action,   # function for updating the controller memory and calculating the control action
                                 cellmodel_auxil.x0_from_init_conds(init_conds,
                                                                    par,
                                                                    synth_genes, synth_miscs, controller_dynvars,
                                                                    modules_name2pos,
                                                                    controller_name2pos),   # initial condition VECTOR
                                 jnp.array(controller_memo0),  # initial controller memory record
                                 u0,    # initial control action, applied before any measurement-informed actions reach the system
                                 (len(synth_genes), len(module1_genes), len(module2_genes)),    # number of synthetic genes
                                 (len(synth_miscs), len(module1_miscs), len(module2_miscs)),    # number of miscellaneous species
                                 modules_name2pos, controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                                 cellmodel_auxil.synth_gene_params_for_jax(par, synth_genes),   # synthetic gene parameters in jax.array form
                                 tf, meastimestep,  # simulation time frame and measurement time step
                                 control_delay,  # delay before control action reaches the system
                                 us_size,  # size of the control action record needed
                                 [0.0], ref_switcher,  # reference values and reference switcher
                                 )
    # record the OFP level
    ofp_levels_for_u_range[i] = np.array(xs_jnp)[-1, modules_name2pos['p_ofp']]

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [4]:
# PLOT THE RESULTS
precbc_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_type="log",
    x_axis_label="u, probe induction",
    y_axis_label="p_ofp, controlled variable",
    x_range=(min(u_range), max(u_range)),
    y_range=(0.9*min(ofp_levels_for_u_range), 1.1*max(ofp_levels_for_u_range)),
    title='Control action vs controlled variable',
    tools="box_zoom,pan,hover,reset"
)
# plot the controlled variable vs control action
precbc_fig.line(x=u_range, y=ofp_levels_for_u_range, line_width=1.5, 
             line_color='black', line_dash='dashed', 
             legend_label='true steady states')
precbc_fig.scatter(x=u_range, y=ofp_levels_for_u_range,
                marker='circle', size=5, 
               color='black', legend_label='true steady states')
# legend formatting
precbc_fig.legend.label_text_font_size = "8pt"
precbc_fig.legend.location = "top_right"
precbc_fig.legend.click_policy = 'hide'

# show plot
bkplot.show(precbc_fig)

In [5]:
# SIMULATE CBC - SET UP THE MODEL

# set up the CBC model
# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
cellmodel_par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(cellmodel_par)  # get default initial conditions

# add reference tracker switcher
cellmodel_par_with_refswitch, ref_switcher = cellmodel_auxil.add_reference_switcher(cellmodel_par,   # cell model parameters
                                                                               refsws.timed_switching_initialise,    # function initialising the reference switcher
                                                                               refsws.timed_switching_switch   # function switching the references to be tracked
                                                                               )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_eff_mRNA, module2_eff_mRNA, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_and_each, synth_miscs_total_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar,\
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = cellmodel_auxil.add_modules_and_controller(
        # module 1
        gms.constfp_initialise,  # function initialising the circuit
        gms.constfp_ode,  # function defining the circuit ODEs
        gms.constfp_F_calc,
        gms.constfp_eff_mRNA,
        # module 2
        gms.cicc_initialise,  # function initialising the circuit
        gms.cicc_ode,  # function defining the circuit ODEs
        gms.cicc_F_calc,
        gms.cicc_eff_mRNA,
        # controller
        ctrls.bangbangchem_initialise,  # function initialising the controller
        ctrls.bangbangchem_action,  # function calculating the controller action
        ctrls.bangbangchem_ode,  # function defining the controller ODEs
        ctrls.bangbangchem_update,  # function updating the controller based on measurements
        # ctrls.pichem_initialise,  # function initialising the controller
        # ctrls.pichem_action,  # function calculating the controller action
        # ctrls.pichem_ode,  # function defining the controller ODEs
        # ctrls.pichem_update,  # function updating the controller based on measurements
        # cell model parameters and initial conditions
        cellmodel_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes= synth_genes_total_and_each[0]
module1_genes = synth_genes_total_and_each[1]
module2_genes = synth_genes_total_and_each[2]
synth_miscs= synth_miscs_total_and_each[0]
module1_miscs = synth_miscs_total_and_each[1]
module2_miscs = synth_miscs_total_and_each[2]
    
# set the circuit parameters common for all cases
par.update(circ_par)

In [6]:
# SIMULATE CBC - SET CONTROLLER AND SIMULATION PARAMETERS

controller_ctrledvar='p_ofp'

# reference p_ofp levels
# num_cbc_refs = 5
# cbc_refs=np.linspace(0, max(ofp_levels_for_u_range), 10)
cbc_refs=ofp_levels_for_u_range
num_cbc_refs=len(cbc_refs)

# simulation parameters
tf = (0.0, 48.0)  # simulation time
meastimestep = 0.1  # hours

# switching between references
par['t_switch_ref']=(tf[1]-tf[0])/num_cbc_refs

# bang-bang controller parameters
par['inducer_level_on']=100.0
par['on_when_below_ref']=False

# PI controller parameters - gains negative as more probe induction => less p_ofp
par['Kp'] = -0.01
par['Ki'] = 0

control_delay=0.01

In [7]:
# SIMULATE CBC - RUN THE SIMULATION

# set up the ODE solver
ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                  control_delay=control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=1e-5)

# solve ODE
timer= time.time()
ts_jnp, xs_jnp,\
    ctrl_memorecord_jnp, uexprecord_jnp, \
    refrecord_jnp  = ode_sim(par,   # model parameters
                             ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                             odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                             controller_ctrledvar,    # name of the variable read and steered by the controller
                             controller_update, controller_action,   # function for updating the controller memory and calculating the control action
                             cellmodel_auxil.x0_from_init_conds(init_conds,
                                                                par,
                                                                synth_genes, synth_miscs, controller_dynvars,
                                                                modules_name2pos,
                                                                controller_name2pos),   # initial condition VECTOR
                             controller_memo0,  # initial controller memory record
                             u0,    # initial control action, applied before any measurement-informed actions reach the system
                             (len(synth_genes), len(module1_genes), len(module2_genes)),    # number of synthetic genes
                             (len(synth_miscs), len(module1_miscs), len(module2_miscs)),    # number of miscellaneous species
                             modules_name2pos, controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                             cellmodel_auxil.synth_gene_params_for_jax(par, synth_genes),   # synthetic gene parameters in jax.array form
                             tf, meastimestep,  # simulation time frame and measurement time step
                             control_delay,  # delay before control action reaches the system
                             us_size,  # size of the control action record needed
                             cbc_refs, ref_switcher,  # reference values and reference switcher
                             )

# convert simulation results to numpy arrays
ts = np.array(ts_jnp)
xs = np.array(xs_jnp)
ctrl_memorecord = np.array(ctrl_memorecord_jnp)
uexprecord = np.array(uexprecord_jnp)
refrecord= np.array(refrecord_jnp)

In [8]:
# PLOT - SYNTHETIC CIRCUITS AND CONTROLLER
# plot synthetic circuit concentrations
mRNA_fig, prot_fig, misc_fig = cellmodel_auxil.plot_circuit_concentrations(ts, xs,
                                                                          par, synth_genes, synth_miscs,
                                                                          modules_name2pos,
                                                                          modules_styles)  # plot simulation results
# plot synthetic circuit regulation functions
F_fig = cellmodel_auxil.plot_circuit_regulation(ts, xs, # time points and state vectors
                                                ctrl_memorecord, uexprecord,    # controller memory and experienced control actions records
                                                refrecord,  # reference tracker records
                                                module1_F_calc, module2_F_calc, # transcription regulation functions for both modules
                                                controller_action, # control action calculator
                                                par, # model parameters
                                                synth_genes_total_and_each,     # list of synthetic genes - total and for each module
                                                synth_miscs_total_and_each,     # list of synthetic miscellaneous species - total and for each module
                                                modules_name2pos,  # dictionary mapping gene names to their positions in the state vector
                                                module1_eff_mRNA, module2_eff_mRNA, # calculating effective synthetic gene mRNA concentrations for each module
                                                controller_name2pos, # dictionary mapping controller species to their positions in the state vector
                                                modules_styles)  # plot simulation results
# plot controller memory, dynamic variables and actions
ctrl_ref_fig, ctrl_memo_fig, ctrl_dynvar_fig, ctrl_u_fig = cellmodel_auxil.plot_controller(ts, xs,
                                                                                        ctrl_memorecord, uexprecord, # controller memory and experienced control actions records
                                                                                        refrecord, # reference tracker records
                                                                                        controller_memos, controller_dynvars,
                                                                                        controller_ctrledvar,
                                                                                        controller_action, controller_update,
                                                                                        par,
                                                                                        synth_genes, synth_miscs,
                                                                                        modules_name2pos,
                                                                                        module1_eff_mRNA, module2_eff_mRNA,
                                                                                        controller_name2pos,
                                                                                        controller_styles,
                                                                                        u0, control_delay)
bkplot.show(bklayouts.grid([[mRNA_fig, prot_fig, misc_fig],
                            [F_fig, None, ctrl_ref_fig],
                            [ctrl_memo_fig, ctrl_dynvar_fig, ctrl_u_fig]]))

In [11]:
# PLOT - HOST CELL DYNAMICS
mass_fig = cellmodel_auxil.plot_protein_masses(ts, xs, par, synth_genes, synth_miscs, modules_name2pos)  # plot simulation results
nat_mrna_fig, nat_prot_fig, nat_trna_fig, h_fig = cellmodel_auxil.plot_native_concentrations(ts, xs,
                                                                                             par,
                                                                                             synth_genes,
                                                                                             synth_miscs,
                                                                                             modules_name2pos)  # plot simulation results
l_figure, e_figure, Fr_figure, ppGpp_figure, nu_figure, D_figure = cellmodel_auxil.plot_phys_variables(ts,
                                                                                                       xs,
                                                                                                       par,
                                                                                                       synth_genes,
                                                                                                       synth_miscs,
                                                                                                       modules_name2pos,
                                                                                                       module1_eff_mRNA, module2_eff_mRNA)  # plot simulation results
bkplot.show(bklayouts.grid([[mass_fig, nat_mrna_fig, nat_prot_fig],
                            [nat_trna_fig, h_fig, l_figure],
                            [e_figure, Fr_figure, ppGpp_figure]]))

In [12]:
# PROCESS THE CBC TRAJECTORY

# gradient relative tolerance for steady state
grad_rtol=0.001


# go through the trajectory and find average p_ofp in the last hour of tracking each reference
u_steady_states=[]
ofp_steady_states=[]
ref_cntr=0
for i in range(0, len(ts)-1):
    if(refrecord[i+1]!=refrecord[i]):      
        # find the time points for the last hour of tracking
        last_hour_indices=np.where((ts>=ts[i]-1) & (ts<=ts[i]))[0]
        
        # get the mean u and p_ofp in the last hour of tracking
        u_mean=np.mean(uexprecord[last_hour_indices])
        ofp_mean=np.mean(xs[last_hour_indices, modules_name2pos['p_ofp']])
        
        # before recording the value, check if the reference is somewhat steady
        grad, _ = np.polyfit(ts[last_hour_indices],
                             xs[last_hour_indices,modules_name2pos['p_ofp']],
                             deg=1)  # get a linear fit for controlled variable vs time
        # if check is passed, record the steady state
        if(abs(grad/ofp_mean)<grad_rtol):
            u_steady_states.append(u_mean)
            ofp_steady_states.append(ofp_mean)    
        
        # move to the next reference
        ref_cntr+=1
        
# add the last reference
last_hour_indices=np.where((ts>=ts[-1]-1) & (ts<=ts[-1]))[0]

# get the mean u and p_ofp in the last hour of tracking
u_mean=np.mean(uexprecord[last_hour_indices])
ofp_mean=np.mean(xs[last_hour_indices, modules_name2pos['p_ofp']])

# before recording the value, check if the reference is somewhat steady
grad, _ = np.polyfit(ts[last_hour_indices],
                     xs[last_hour_indices,modules_name2pos['p_ofp']],
                     deg=1)  # get a linear fit for controlled variable vs time
# if check is passed, record the steady state
if(abs(grad/ofp_mean)<grad_rtol):
    u_steady_states.append(u_mean)
    ofp_steady_states.append(ofp_mean)
    
# convert to numpy arrays
u_steady_states=np.array(u_steady_states)
ofp_steady_states=np.array(ofp_steady_states)

In [14]:
# PLOT BIFURCATION DIAGRAM

# get the calculated control actions
u_calc = np.zeros(len(ts))
for i in range(0, len(ts)):
    u_calc[i] = controller_action(ts[i], xs[i, :], ctrl_memorecord[i], refrecord[i], par,
                                  modules_name2pos, controller_name2pos, 'p_ofp')
# for convenience of display, zero action shown as minimal action
u_calc[u_calc==0]=min(u_range)
uexprecord[uexprecord==0]=min(u_range)
    
cbc_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_type="log",
    x_axis_label="u, probe induction",
    y_axis_label="p_ofp, controlled variable",
    x_range=(min(u_range), max(u_range)),
    y_range=(0.9 * min(ofp_levels_for_u_range), 1.1 * max(ofp_levels_for_u_range)),
    title='Control action vs controlled variable',
    tools="box_zoom,pan,hover,reset"
)
# plot the controlled variable vs control action
cbc_fig.line(x=u_range, y=ofp_levels_for_u_range, line_width=1.5,
            line_color='black', line_dash='dashed',
            legend_label='true steady states')
cbc_fig.scatter(x=u_range, y=ofp_levels_for_u_range,
               marker='circle', size=7.5,
               color='black', legend_label='true steady states')
# legend formatting
cbc_fig.legend.label_text_font_size = "8pt"
cbc_fig.legend.location = "top_right"
cbc_fig.legend.click_policy = 'hide'

# plot the cbc trajectory for calculated control actions
cbc_fig.line(x=u_calc, y=xs[:, modules_name2pos['p_ofp']], line_width=1.5, 
             line_color='blue', line_alpha=0.5,
             legend_label='cbc (calculated u)')
# plot the cbc trajectory for experienced control actions
# cbc_fig.line(x=uexprecord, y=xs[:, modules_name2pos['p_ofp']], line_width=1.5, 
#              line_color='red', line_alpha=0.5,
#              legend_label='cbc (experienced u)')

# plot the CBC 'steady states'
cbc_fig.scatter(x=u_steady_states, y=ofp_steady_states,
                marker='circle', size=7.5, 
               color='violet', legend_label='cbc steady states')

# show plot
bkplot.show(cbc_fig)